Import All Necessary Files

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import pandas as pd
import bitsandbytes
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

Check for GPU availability

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Load the pre-trained model and tokenizer

In [ ]:
MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    trust_remote_code=True  # Allow loading custom tokenizer implementations
)

Ensure pad token is set if not present

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Data Processing

In [ ]:
# Load dataset (modify path if using a local dataset)
dataset = load_dataset("your_dataset_name_here")

Define a custom dataset class for tokenization

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {key: val.squeeze(0) for key, val in encoded.items()}

# Preprocess dataset
dataset = dataset.map(lambda example: {"text": example["column_name"]})  # Replace 'column_name' with actual column name

# Split dataset
dataset = dataset["train"].train_test_split(test_size=0.1)

train_dataset = CustomDataset(dataset["train"]["text"], tokenizer)
test_dataset = CustomDataset(dataset["test"]["text"], tokenizer)

Define LoRA configuration for efficient training

In [ ]:
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM"
)

Load model and apply LoRA tuning

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model = get_peft_model(model, lora_config)
model.to(device)

Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",  # Directory for saving checkpoints
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"  # Disable logging to external services
)

Data collator for dynamic padding

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if using masked language modeling
)

Trainer setup

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

Train the model

In [ ]:
trainer.train()

Save the fine-tuned model

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Fine-tuning completed and model saved successfully!")